<a href="https://colab.research.google.com/github/tomonari-masada/course2023-sml/blob/main/09_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 糖尿病をロジスティック回帰で予測してみる

* 有名なPima Indians Diabetes Databaseを使う（下リンク先）

 * https://www.kaggle.com/uciml/pima-indians-diabetes-database

* ロジスティック回帰、そして、分類の評価については、下記も参照
 * https://developers.google.com/machine-learning/crash-course/logistic-regression/
 * https://developers.google.com/machine-learning/crash-course/classification/

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_auc_score, auc, roc_curve
from sklearn.model_selection import StratifiedKFold

%config InlineBackend.figure_format = 'retina'

## 1) データの読み込み

In [ ]:
diabetes = pd.read_csv('/content/drive/MyDrive/data/diabetes.csv')

In [ ]:
diabetes.head()

In [ ]:
y = diabetes['Outcome']
X = diabetes.drop('Outcome', axis=1)

## 2) 訓練データ、テストデータに分割

**この分割は変えないようにしてください。**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=123)

In [ ]:
X_train.describe()

In [ ]:
X_train.hist(bins=50, figsize=(12,12));

* 以下、訓練データ部分を使って、交差検証によって良いモデルを探す。

---



## 3) デフォルト設定のロジスティック回帰をベースラインとみなしてテストデータでの評価値を得る
* 交差検証も何もせずに、単にテストセット以外の部分で、モデルの学習を実行する。

In [ ]:
baseline = LogisticRegression(random_state=123)
baseline.fit(X_train, y_train)

* `max_iter`が小さいとの警告が出ているので、増やして学習しなおし。

In [ ]:
baseline = LogisticRegression(max_iter=1000, random_state=123)
baseline.fit(X_train, y_train)

* 大丈夫だったので、テストデータでの最終評価値を得る。

In [ ]:
print(f'test score: {baseline.score(X_test, y_test):.4f}')

* Area under ROC curveも計算してみる。


In [ ]:
y_test_pred_proba = baseline.predict_proba(X_test)
print(f'ROC AUC: {roc_auc_score(y_test, y_test_pred_proba[:,1]):.4f}')

* ROC curveを描いてみる。
 * https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html#sphx-glr-auto-examples-model-selection-plot-roc-py

In [ ]:
def draw_roc_curve(model, X_test, y_test):
  y_score = model.decision_function(X_test)

  fpr, tpr, _ = roc_curve(y_test, y_score)
  roc_auc = auc(fpr, tpr)

  plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.4f})')
  plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.0])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic example')
  plt.legend(loc="lower right")

In [ ]:
draw_roc_curve(baseline, X_test, y_test);

* これをベースラインとみなす。
* これより良い結果を得るべく、試行錯誤する。
* 試行錯誤した結果として辿り着いたモデルで、最後に一回、テストデータ上での評価を行う。

* ロジスティック回帰についてscoreがどのように計算されているかの確認
 * thresholdが0.5である必要は、実は、ない。
 * thresholdを、交差検証で決定してもよい。

* `threshold = 0.5`とすれば、次のセルで求まる値と、上で求めたtest scoreは、一致する。

In [ ]:
threshold = 0.5
n_correct_answers = ((baseline.predict_proba(X_test)[:,1] >= threshold) * 1 == y_test).sum()
print(f'test score at a threshold {threshold}: {n_correct_answers / len(y_test):.4f}')

## 4) 交差検証しつつ試行錯誤する

* 元々の訓練データのコピーを作っておく。

In [ ]:
X_train_original = X_train.copy()
X_test_original = X_test.copy()

### A) 交差検証の準備

In [ ]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

### B) 交差検証のための関数

In [ ]:
def cv(skf, X_train, y_train, preprocess=None, **kwargs):

  # キーワード引数として、モデルの設定を指定できるようにしてある。
  for kwarg in kwargs:
    print(f'{kwarg} = {kwargs[kwarg]}')

  # 交差検証のループ
  scores = list()
  for train_index, valid_index in skf.split(X_train, y_train):

    cv_X_train = X_train.iloc[train_index]
    cv_y_train = y_train.iloc[train_index]
    cv_X_valid = X_train.iloc[valid_index]
    cv_y_valid = y_train.iloc[valid_index]

    # データの前処理
    #   その都度、関数preprocessを定義してから、この関数cvを呼び出す。
    if preprocess:
      cv_X_train, cv_X_valid = preprocess(cv_X_train, cv_X_valid)

    # ロジスティック回帰の学習
    if not 'max_iter' in kwargs:
      model = LogisticRegression(**kwargs, max_iter=1000)
    else:
      model = LogisticRegression(**kwargs)
    model.fit(cv_X_train, cv_y_train)

    # 検証データでの評価
    score = model.score(cv_X_valid, cv_y_valid)
    print(f'score: {score:.4f}')
    scores.append(score)

  mean_score = np.array(scores).mean()
  print(f'mean score: {mean_score:.4f}')
  return mean_score

### C) デフォルトの設定での評価
* 交差検証で性能評価するとどうなるかを確認している。

In [ ]:
cv(skf, X_train, y_train);

### D) BloodPressureへの対応

* まず、属性「BloodPressure」について、ヒストグラムを描いてよくよく眺める。


In [ ]:
sns.histplot(X_train['BloodPressure']);

* 0という値がけっこうあるらしい。実は、これは欠測値。そこで、中央値で埋めることにする。

In [ ]:
X_train_copy = X_train.copy()

feature = 'BloodPressure'
imp = SimpleImputer(missing_values=0, strategy='median')
X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

sns.histplot(X_train_copy[feature]);

* 交差検証で評価する。

In [ ]:
def preprocess(X_train, X_valid):
  imp = SimpleImputer(missing_values=0, strategy='median')

  X_train_copy = X_train.copy()
  X_valid_copy = X_valid.copy()

  feature = 'BloodPressure'
  X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
  X_valid_copy[feature] = imp.transform(X_valid[[feature]])
  print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

  return X_train_copy, X_valid_copy

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

* 下のようにテストセット以外全体で欠損値を埋めても、結果は変わらない。

In [ ]:
X_train_copy = X_train.copy()

feature = 'BloodPressure'
imp = SimpleImputer(missing_values=0, strategy='median')
X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

cv(skf, X_train_copy, y_train);

### E) BMIへの対応

* 次に、training dataの「BMI」のヒストグラムを描いてみる


In [ ]:
sns.histplot(X_train['BMI']);

* やはり欠測値の部分が0とされているようなので、先ほどと同様、中央値で埋める。


In [ ]:
X_train_copy = X_train.copy()

feature = 'BMI'
imp = SimpleImputer(missing_values=0, strategy='median')
X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

sns.histplot(X_train_copy[feature]);

* 交差検証で評価する。

In [ ]:
def preprocess(X_train, X_valid):
  imp = SimpleImputer(missing_values=0, strategy='median')

  X_train_copy = X_train.copy()
  X_valid_copy = X_valid.copy()

  for feature in ['BloodPressure', 'BMI']:
    X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
    X_valid_copy[feature] = imp.transform(X_valid[[feature]])
    print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

  return X_train_copy, X_valid_copy

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

* 下のようにテストセット以外全体で欠損値を埋めても、結果は変わらない。

In [ ]:
X_train_copy = X_train.copy()

for feature in ['BloodPressure', 'BMI']:
  imp = SimpleImputer(missing_values=0, strategy='median')
  X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
  print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

cv(skf, X_train_copy, y_train);

### F) Glucoseへの対応

In [ ]:
sns.histplot(X_train['Glucose']);

In [ ]:
X_train_copy = X_train.copy()

feature = 'Glucose'
imp = SimpleImputer(missing_values=0, strategy='median')
X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

sns.histplot(X_train_copy[feature]);

In [ ]:
def preprocess(X_train, X_valid):
  imp = SimpleImputer(missing_values=0, strategy='median')

  X_train_copy = X_train.copy()
  X_valid_copy = X_valid.copy()

  for feature in ['BloodPressure', 'BMI', 'Glucose']:
    X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
    X_valid_copy[feature] = imp.transform(X_valid[[feature]])
    print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

  return X_train_copy, X_valid_copy

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

* 下のようにテストセット以外全体で欠損値を埋めても、結果は変わらない。

In [ ]:
X_train_copy = X_train.copy()

for feature in ['BloodPressure', 'BMI', 'Glucose']:
  imp = SimpleImputer(missing_values=0, strategy='median')
  X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
  print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

cv(skf, X_train_copy, y_train);

* ここまでの交差検証でのベスト・スコアは0.7759。

* 訓練データと検証データを区別して欠損値を埋めても、テストセット以外全体で欠損値を埋めても、結果はあまり変わらなさそう。
* 以下、単純に、テストセット以外全体で欠損値を埋めることで、ベストな前処理を探ることにする。

* X_trainの欠損値を埋めて、元のX_trainを上書きする。

In [ ]:
X_train_copy = X_train.copy()

for feature in ['BloodPressure', 'BMI', 'Glucose']:
  imp = SimpleImputer(missing_values=0, strategy='median')
  X_train_copy[feature] = imp.fit_transform(X_train[[feature]])
  print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

X_train = X_train_copy

cv(skf, X_train, y_train);

### G) SkinThicknessとInsulinへの対応

In [ ]:
sns.histplot(X_train['SkinThickness'], bins=50);

In [ ]:
sns.histplot(X_train['Insulin'], bins=50);

In [ ]:
(X_train['SkinThickness'] == 0).sum()

In [ ]:
(X_train['Insulin'] == 0).sum()

* 欠測値が多すぎるので、同じ一つの値で埋めると、問題あり。

In [ ]:
((X_train['SkinThickness'] == 0) & (X_train['Insulin'] == 0)).sum()

In [ ]:
for i in X_train.index[X_train['SkinThickness'] == 0]:
  if not i in X_train.index[X_train['Insulin'] == 0]:
    print('No')

* SkinThicknessが0の個体は、必ずInsulinも0になっているらしい。

 * ただし、これは訓練データだけでこうなっているだけかもしれないので、この事実に依存して何かをすることはしない。

* 線形回帰でSkinThicknessとInsulinの欠測部分を埋める。
 * 欠測部分を同じ値で埋めたくないため。

In [ ]:
# 欠測値を埋めるための回帰で、特徴量として使う列
columns = X_train.columns.drop('SkinThickness').drop('Insulin')

In [ ]:
def preprocess(X_train, X_valid):

  X_train_copy = X_train.copy()
  X_valid_copy = X_valid.copy()

  for feature in ['SkinThickness', 'Insulin']:
    reg = LinearRegression()
    indices = (X_train[feature] != 0)
    reg.fit(X_train.loc[indices, columns], X_train.loc[indices, feature])
    X_train_copy.loc[~ indices, feature] = reg.predict(X_train.loc[~ indices, columns])

    indices = (X_valid[feature] != 0)
    X_valid_copy.loc[~ indices, feature] = reg.predict(X_valid.loc[~ indices, columns])

  return X_train_copy, X_valid_copy

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

* 悪くなったので不採用。

* 次は、k-NNを使って欠測値を埋める。

In [ ]:
# columnsとkは、関数の外から値を指定する。

def preprocess(X_train, X_valid):
  print(f'imputation k-NN k={k}')

  X_train_copy = X_train.copy()
  X_valid_copy = X_valid.copy()

  for feature in ['SkinThickness', 'Insulin']:
    reg = KNeighborsRegressor(n_neighbors=k)
    indices = (X_train[feature] != 0)
    reg.fit(X_train.loc[indices, columns], X_train.loc[indices, feature])
    X_train_copy.loc[~ indices, feature] = reg.predict(X_train.loc[~ indices, columns])

    indices = (X_valid[feature] != 0)
    X_valid_copy.loc[~ indices, feature] = reg.predict(X_valid.loc[~ indices, columns])

  return X_train_copy, X_valid_copy

In [ ]:
best_k, best_score = 1, 0.0

for k in range(1, 21):
  score = cv(skf, X_train, y_train, preprocess=preprocess)
  print('-'*64)
  if best_score < score:
    best_k, best_score = k, score

print(f'best score {best_score:.4f} for k = {best_k}')

* 選ばれたkの値を使って、訓練データ全体で欠測値を埋める。

In [ ]:
k = best_k

X_train_copy = X_train.copy()

for feature in ['SkinThickness', 'Insulin']:
  reg = KNeighborsRegressor(n_neighbors=k)
  indices = (X_train[feature] != 0)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, feature])
  X_train_copy.loc[~ indices, feature] = reg.predict(X_train.loc[~ indices, columns])

In [ ]:
sns.histplot(X_train_copy['SkinThickness'], bins=50);

In [ ]:
sns.histplot(X_train_copy['Insulin'], bins=50);

In [ ]:
cv(skf, X_train_copy, y_train);

* 分類性能が良くなったので、埋めた後のデータセットを採用する。

In [ ]:
X_train = X_train_copy

* Pregnanciesを除いて、値0は無くなっている。

In [ ]:
(X_train == 0).sum()

### H) スケーラー

In [ ]:
def preprocess(X_train, X_valid):
  scaler = MinMaxScaler()
  scaler.fit(X_train)
  return scaler.transform(X_train), scaler.transform(X_valid)

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

In [ ]:
def preprocess(X_train, X_valid):
  scaler = StandardScaler()
  scaler.fit(X_train)
  return scaler.transform(X_train), scaler.transform(X_valid)

In [ ]:
cv(skf, X_train, y_train, preprocess=preprocess);

いずれも不採用。

### I) 正則化

In [ ]:
best_C, best_score = 0, 0

for C in np.power(10.0, np.arange(13) - 5):
  score = cv(skf, X_train, y_train, C=C)
  if best_score < score:
    best_C, best_score = C, score
  print('-' * 64)

print(f'best score {best_score:.4f} for C={C}')

* Cの値が大きいほど良いということは、正則化なしが良いということ。

In [ ]:
cv(skf, X_train, y_train, penalty='none');

* 以上をまとめると・・・
 * 'BloodPressure', 'BMI', 'Glucose'の欠損値は中央値で埋める。
 * 'SkinThickness', 'Insulin'の欠損値はk-NNで埋める。
 * ロジスティック回帰は正則化なしで使う。

## 5) テストデータで最終評価

* 訓練データの中央値を使って、テストデータの欠測値を埋める。

In [ ]:
# 訓練データについては、最初に取っておいたオリジナル X_train_original を使うこと。

X_train_copy = X_train_original.copy()
X_test_copy = X_test.copy()

for feature in ['BloodPressure', 'BMI', 'Glucose']:
  imp = SimpleImputer(missing_values=0, strategy='median')
  X_train_copy[feature] = imp.fit_transform(X_train_original[[feature]])
  X_test_copy[feature] = imp.transform(X_test[[feature]])
  print(f'imputation fill value for {feature}: {imp.statistics_[0]}')

* k-NNでは、上で'BloodPressure', 'BMI', 'Glucose'の欠測値を埋めたデータを使う。

In [ ]:
X_train = X_train_copy
X_test = X_test_copy

* 訓練データで近傍を見つけることによって、テストデータの欠測値を埋める。

In [ ]:
k = best_k

X_train_copy = X_train.copy()
X_test_copy = X_test.copy()

for feature in ['SkinThickness', 'Insulin']:
  reg = KNeighborsRegressor(n_neighbors=k)
  indices = (X_train[feature] != 0)
  reg.fit(X_train.loc[indices, columns], X_train.loc[indices, feature])
  X_train_copy.loc[~ indices, feature] = reg.predict(X_train.loc[~ indices, columns])
  indices = (X_test[feature] != 0)
  X_test_copy.loc[~ indices, feature] = reg.predict(X_test.loc[~ indices, columns])

In [ ]:
X_train = X_train_copy
X_test = X_test

In [ ]:
model = LogisticRegression(max_iter=1000, penalty='none', random_state=123)
model.fit(X_train, y_train)
print('test score: {:.4f}'.format(model.score(X_test, y_test)))

In [ ]:
y_test_pred_proba = model.predict_proba(X_test)
print('ROC AUC: {:.4f}'.format(roc_auc_score(y_test, y_test_pred_proba[:,1])))

In [ ]:
y_score = baseline.decision_function(X_test_original) # ベースラインには元のテストデータを使う
fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)

y_score_ours = model.decision_function(X_test)
fpr_ours, tpr_ours, _ = roc_curve(y_test, y_score_ours)
roc_auc_ours = auc(fpr_ours, tpr_ours)

plt.plot(fpr, tpr, color='darkorange', label=f'ROC curve (area = {roc_auc:.4f})')
plt.plot(fpr_ours, tpr_ours, color='firebrick', label=f'ROC curve (area = {roc_auc_ours:.4f})')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right");

# 2022/06/11の課題
* 上の結果を改良できるかどうか、試行錯誤してみてください。